In [ ]:
from langchain.chat_models import ChatOpenAI


message = "내일 날씨좀 알려줘."
llm = ChatOpenAI(
  model_name="gpt-4o-mini",
)

response = llm.predict(message)

print(response)

In [ ]:
from langchain.chat_models import ChatOpenAI


chat = ChatOpenAI(
  model_name="gpt-4o-mini",
  temperature=0.1 # 낮을수록 정확도가 높고, 높을수록 창의성이 높아짐
)


from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
  SystemMessage(content="You are a geography expert. And you only reply in Italian."),
  AIMessage(content="Ciao, mi chiamo Paolo!"),
  HumanMessage(content="What is the distance between Mexico and Thailand. Also, what is your name?")
]

chat.predict_messages(messages)


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

template = PromptTemplate(
  input_variables=["country_a", "country_b"],
  template="What is the distance between {country_a} and {country_b}",
)
chat = ChatOpenAI(
  model_name="gpt-4o-mini",
  temperature=0.1
)

prompt = template.format(country_a="Mexico", country_b="Thailand")

chat.predict(prompt)

In [ ]:
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(
  model_name="gpt-4o-mini",
  temperature=0.1
)

template = ChatPromptTemplate.from_messages([
  ("system", "You are a geography expert. And you only reply in {language}"),
  ("ai", "Ciao, mi chiamo {name}"),
  ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?")
])

prompt = template.format_messages(
  language="Greek",
  name="Socrates",
  country_a="Mexico",
  country_b="Thailand"
)

chat.predict_messages(prompt)


In [ ]:
from langchain.schema import BaseOutputParser


class CommonOutputParser(BaseOutputParser):
  def parse(self, text):
    items = text.strip().split(",")
    return list(map(str.strip, items))
  
  
p = CommonOutputParser()
  
p.parse("Hello,how,are,you")

ChatPromptTemplate

In [ ]:


template = ChatPromptTemplate.from_messages(
  [
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated a list of {max_items}. Do not reply with anything else."),
    ("human", "{question}")
  ]
)

prompt = template.format_messages(
  max_items=10,
  question="What are the planets?"
)

result = chat.predict_messages(prompt)

p = CommonOutputParser()

p.parse(result.content)



In [ ]:
chain = template | chat | CommonOutputParser()

chain.invoke({
  "max_items": 5,
  "question": "What are the pokemons?"
})

In [18]:



chef_template = ChatPromptTemplate.from_messages([
  ("system", "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
  ("human", "i want to cook {cuisine} food.")
])

chef_chain = chef_template | chat

veg_chef_prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a vegetarian chef speciailized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
  ("human", "{recipe}")
])


veg_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({
  "cuisine": "Indian"
})

AIMessage(content="Chana Masala is a fantastic vegetarian dish that is already suitable for a vegetarian diet, as it primarily consists of chickpeas and spices. However, if you're looking for alternatives or modifications to enhance the recipe, here are a few suggestions:\n\n### Chana Masala Recipe (Vegetarian)\n\n#### Ingredients:\n- 1 can (15 oz) chickpeas (garbanzo beans), drained and rinsed\n- 1 medium onion, finely chopped\n- 2 medium tomatoes, chopped\n- 2-3 cloves garlic, minced\n- 1-inch piece of ginger, grated\n- 2 green chilies, slit (adjust to taste)\n- 1 teaspoon cumin seeds\n- 1 teaspoon coriander powder\n- 1 teaspoon cumin powder\n- 1 teaspoon garam masala\n- ½ teaspoon turmeric powder\n- ½ teaspoon red chili powder (adjust to taste)\n- Salt to taste\n- 2 tablespoons vegetable oil (can substitute with olive oil or coconut oil)\n- Fresh cilantro, chopped (for garnish)\n- Lemon wedges (for serving)\n\n#### Instructions:\n\n1. **Heat Oil**: In a large pan, heat the vegetable

In [23]:

from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
  model_name="gpt-4o-mini",
  temperature=0.1,
  callbacks=[StreamingStdOutCallbackHandler()],
  streaming=True
)

poetry_prompt = ChatPromptTemplate.from_messages({
  ("system", "너는 세상에서 가장 아름답고 멋진 시를 만드는 시인이야. 너는 프로그래밍 세상에서 특정 언어에 대한 시를 쓸 수 있어."),
  ("human", "{language}")
})

poetry_chain = poetry_prompt | chat

poetry_chain.invoke({
  "language": "Java"
})

poetry_description_prompt = ChatPromptTemplate.from_messages({
  ("system", "너는 시인의 시를 보고 설명하는 해설가야."),
  ("human", "{poetry}")
})

poetry_description_chain = poetry_description_prompt | chat

final_chain = {"poetry": poetry_chain} | poetry_description_chain


**Java의 노래**

자바의 세계, 그 깊은 바다,  
객체의 춤, 메서드의 노래,  
클래스와 인터페이스, 조화의 선율,  
코드의 미로 속, 우리는 항해하네.

강한 타입, 안전한 품,  
예외 처리로 지키는 꿈,  
멀티스레드의 리듬 속에,  
동시성의 마법, 우린 함께해.

JVM 위에서 펼쳐지는,  
플랫폼의 경계를 넘는 자유,  
"Hello, World!"의 첫 인사,  
모든 시작은 이곳에서.

컬렉션의 바다, 리스트의 물결,  
맵의 길을 따라, 데이터의 향연,  
람다의 숨결, 함수형의 미소,  
코드의 아름다움, 그 속에 담겨.

자바의 철학, "Write Once, Run Anywhere",  
어디서든 빛나는 그 가능성,  
우리는 함께 만들어가는,  
이 세상의 모든 꿈과 희망.

자바여, 너의 이름을 부르며,  
코드의 시를, 영원히 노래하리,  
우리가 만든 이 아름다운 세계,  
자바의 노래, 끝없이 이어지리.